In diesem Notebook werden wird der Code gesammelt, der für die finale Abgabe der Masterarbeit nicht relevant ist, aber dennoch interessant sein könnte. 

In [3]:
import pandas as pd
import json
import os

#eigene Module
from src.dataimport import load_text

# JSON zu Pandas Dataframes
MajorClaims, Claims, Premises und Argumentationsbeziehungen werden in einzelne Dataframes umgewandelt. 

In [ ]:
def json_to_dataframe(json_data):
    data = json.loads(json_data)
    # Create a DataFrame for MajorClaims
    major_claims_df = pd.DataFrame(data["MajorClaims"])
    major_claims_df["Type"] = "MajorClaim"
    # Create a DataFrame for Claims
    claims_df = pd.DataFrame(data["Claims"])
    claims_df["Type"] = "Claim"
    # Create a DataFrame for Premises
    premises_df = pd.DataFrame(data["Premises"])
    premises_df["Type"] = "Premise"
    # Create a DataFrame for ArgumentativeRelations
    relations_df = pd.DataFrame(data["ArgumentativeRelations"])

    # for relations df calculate the types
    relations_df["OriginType"] = relations_df["Origin"].apply(lambda x: x[0])
    relations_df["TargetType"] = relations_df["Target"].apply(lambda x: x[0])

    return major_claims_df, claims_df, premises_df, relations_df

# Example usage
json_data = load_text(transformed_json_files[0])

major_claims_df, claims_df, premises_df, relations_df = json_to_dataframe(json_data)


```python
major_claims_df

ID	Text	Type
0	MC1	we should attach more importance to cooperatio...	MajorClaim
1	MC2	a more cooperative attitudes towards life is m...	MajorClaim


claims_df

ID	Text	Type
0	C1	through cooperation, children can learn about ...	Claim
1	C2	competition makes the society more effective	Claim
2	C3	without the cooperation, there would be no vic...	Claim


premises_df

ID	Text	Type
0	P1	What we acquired from team work is not only ho...	Premise
1	P2	During the process of cooperation, children ca...	Premise
2	P3	All of these skills help them to get on well w...	Premise
3	P4	the significance of competition is that how to...	Premise
4	P5	when we consider about the question that how t...	Premise
5	P6	Take Olympic games which is a form of competit...	Premise


relations_df

Origin	Relation	Target	OriginType
0	C1	For	MC	C
1	P1	supports	C1	P
2	P2	supports	C1	P
3	P3	supports	C1	P
4	C2	Against	MC	C
5	C3	For	MC	C
6	P6	supports	C3	P
7	P5	supports	C3	P
8	P4	supports	C2	P
```

Langchain hat bisher (Stand 12/24) noch keine Möglichkeit die Batch API von OpenAI zu verwenden, sondern nur über die Standard API. Dies geht unter anderem aus dem folgenden Forenbeitrag hervor: https://github.com/langchain-ai/langchain/discussions/21643

Um die Kosten der Anfragen weiter zu reduzieren wurde der ursprüngliche Ansatz über LangChain verworfen.

# Unterteilung in gleichgroße Teile

In [1]:
list = [0, 1, 2, 3, 4, 5, 6, 7]
chunk_size = 4
chunks = [list[i:i + chunk_size] for i in range(0, len(list), chunk_size)]
print(chunks)

for i , chunk in enumerate(chunks):
    output = "\n".join(str(item) for item in chunk)
    print(f"Output {i + 1}:\n{output}")

[[0, 1, 2, 3], [4, 5, 6, 7]]
Output 1:
0
1
2
3
Output 2:
4
5
6
7


# Erstellung einer Batch API Anfrage als JSONL

In [ ]:
def generate_batch_input_split(test_df: pd.DataFrame, prompt_df: pd.DataFrame, file_name: str, num_files: int):
    if num_files <= 0:
        raise ValueError("num_files muss größer als 0 sein.")

    temperature = 0
    llm_seed = 123
    model = "gpt-4o-mini"  

    dict_list = []
    # iteration über Zero-Shot-Prompts
    for _, prompt_row in prompt_df.iterrows():
        # Iteration über Testdaten
        for _, test_df_row in test_df.iterrows():
            custom_id_str = prompt_row['prompt_name'] + "_" + test_df_row['txt_file']# + "_" + str(id_counter)
            # write batch input for jsonl file
            input_dict = {"custom_id": custom_id_str, 
                          "method": "POST", "url": "/v1/chat/completions",
                          "body": {"model": model,
                                   "messages": [{"role": "developer", "content": prompt_row['prompt_txt']}, # system Rolle wurde in developer umbenannt
                                                {"role": "user", "content": "Text: " + test_df_row['txt']}], # user Rolle für Eingaben des Nutzers wie bei ChatGPT 
                                                "temperature": temperature,
                                                "seed": llm_seed,
                                                "response_format": {
                                                    "type": "json_schema", # wichtig festzulegen, da sonst Fehlermeldung
                                                    "json_schema": {
                                                        "name": "ArgumentMiningExtraction", # wichtig festzulegen, da sonst Fehlermeldung
                                                        "schema": response_format, # strukturiertes Output-Format von oben
                                                        "strict": True 
                                                    }
                                                    }
                                                }
                                     }
            dict_list.append(input_dict)

    
    chunk_size = len(dict_list) // num_files + (len(dict_list) % num_files > 0) # Ermittelt die größe der Chunks, indem die Anzahl der Elemente durch die Anzahl der Dateien geteilt wird. Bei einem Rest wird ein Chunk mehr erstellt.
    # Floor-Operator "//" dividiert und rundet auf die nächste ganze Zahl ab (Bsp.: 7 / 2 = 3.5 . 7 // 2 = 3). Modulo Operator "%" gibt den Rest der Division an. Wenn 
    chunks = [dict_list[i:i + chunk_size] for i in range(0, len(dict_list), chunk_size)] # Teilt die Liste in gleich große Teile auf. 
    # Beispiel für 7 Elemente (dict_list) und 2 Dateien (num_files): chunk_size = 7 // 2 + (7 % 2 > 0) = 4. cunks = [dict_list[0:4], dict_list[4:7]]. Intervall [0:4] = 0, 1, 2, 3. Intervall [4:7] = 4, 5, 6. Der letzte Index wird nicht mit eingeschlossen. 

    # Output in JSONL-Dateien schreiben
    for i, chunk in enumerate(chunks): # enumerate iteriert über die Chunks-Liste und gibt den Index und das Element zurück
        jsonl_output = "\n".join(json.dumps(item) for item in chunk) # Schreibt die Elemente in einzelne Zeilen in einen JSONL-String
        with open(f"batch_api/input/{file_name}_{i + 1}.jsonl", 'w') as f: # Schreibt den JSONL-String in eine Datei mit dem übergebenen Dateinamen und der fortlaufender Nummerierung
            f.write(jsonl_output)

    return [f"{file_name}_{i + 1}.jsonl" for i in range(num_files)] # Bezeichnet die Dateien fortlaufend anhand der Anzahl der Dateien


# Quelle Batch API: https://platform.openai.com/docs/guides/batch?lang=python
# Quelle text generation: https://platform.openai.com/docs/guides/text-generation

# Batches unterbrechen

In [ ]:
# cancel_batch = [batch.id for batch in batches_data if batch.metadata["description"] == "Zero-shot prompts with 10 examples from the training set"]
# cancel_batch

# # Cancel the batch
# for batch_id in cancel_batch:
#     client.batches.cancel(batch_id) 

# Übersicht der auf OpenAI Platform hochgeladene Dateien für Client 

In [ ]:
files_data = client.files.list().data
for file in files_data:
    print(f"File-ID: {file.id}")
    print(f"File-Name: {file.filename}")
    print(f"File-Status: {file.status}")
    print(f"-" * 40) 

# auf OpenAI Platform hochgeladene Dateien löschen

In [ ]:
# Löschen einer hochgeladenen Datei
# einzelne Datei
client.files.delete("file-LEPdU2QustjMSEY5e1RSzR")

# Liste von Dateien
del_files_id = [file.id for file in files_data if file.filename.startswith("one-shot")]
#del_files_id

for file_id in del_files_id:
     client.files.delete(file_id)

# Pfad aus Pfad inkl. Dateiname extrahieren

In [4]:
os.path.dirname("batch_api/input/full_batch_input.jsonl")

'batch_api/input'

In [8]:
print(f"{"-"*30}")

------------------------------


# Dataframes als csv speichern und laden

In [ ]:
# csv speichern
df.to_csv('dataframe.csv', index=False)

# dataframe laden
df = pd.read_csv('dataframe.csv')
df.head()